In [58]:
%load_ext autoreload
%autoreload 2
from dataloader import FakenNewsNet
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
dataset = FakenNewsNet("./dataset/FakeNewsNet/politifact/")

./dataset/FakeNewsNet/politifact//real/subgraphs ./dataset/FakeNewsNet/politifact//real/features
politifact100.txt


NameError: name 'load_features' is not defined

In [28]:
tweet_node_list = [x.user_id for x in node_list]

In [24]:
edge_list = load_edge_list("./dataset/FakeNewsNet/politifact/real/subgraphs/politifact100.txt")

In [31]:
full_node_list = set()

for u,v in edge_list:
    if u != 0:
        full_node_list.add(u)
    full_node_list.add(v)

In [34]:
diff_list = full_node_list.difference(tweet_node_list)
len(diff_list)

1962